In [59]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings("ignore")

On charge le jeu de donnèes pré-traité et enregistrer dans un .csv

In [60]:
heart = pd.read_csv("heart-disease-model.csv", index_col='id')

On encode les valeurs qualitattives avec un l'OrdinalEncoder de sklearn 

In [61]:
from sklearn.preprocessing import OrdinalEncoder
encoder = OrdinalEncoder()

heart_qali = heart[["genre", "cholesterol", "fumeur", "conso_alco", "malade", "imc_result", "pression_arterielle"]]


genre = encoder.fit_transform(heart_qali[['genre', 'conso_alco', 'malade', 'fumeur']])

heart_qali[["genre", "conso_alco", "malade", "fumeur"]] = genre
heart_qali[["genre", "conso_alco" ,"malade", "fumeur"]] = heart_qali[["genre", "conso_alco", "malade", "fumeur"]].astype("int")

heart_dummies = pd.get_dummies(heart_qali)

On standardise les valeurs quantitatives avec un StandartScaler()  
On cherche à avoir une moyenne égale à 0 et un écart type égale a 1

In [62]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

heart_quanti = heart[["age", "taille", "Poids", "pression_diasto", "pression_systo", "imc", "age_annee"]] 

scaler.fit(heart_quanti)

heart_scaler = scaler.transform(heart_quanti)

heart_scaled = pd.DataFrame(heart_scaler, columns=heart_quanti.columns)

heart_scaled.describe().T

,count,mean,std,min,25%,50%,75%,max
age,68652.0,-5.829916e-18,1.000007,-3.510250,-0.732173,0.095822,0.753681,1.721430
taille,68652.0,-7.325431e-16,1.000007,-13.363628,-0.655128,0.078054,0.689040,10.464809
Poids,68652.0,-9.919267e-17,1.000007,-4.408275,-0.636029,-0.147034,0.551531,8.794589
pression_diasto,68652.0,1.276896e-15,1.000007,-3.881198,-0.138704,-0.138704,0.930580,4.780003
pression_systo,68652.0,-3.253910e-16,1.000007,-3.416596,-0.396876,-0.396876,0.811012,5.340592
imc,68652.0,-4.983802e-16,1.000007,-3.976705,-0.602237,-0.193500,0.430489,44.845017
age_annee,68652.0,-3.031370e-16,1.000007,-3.445674,-0.786522,0.099862,0.690785,1.724899


On sépare nos data et notre cible

In [69]:
from sklearn.model_selection import train_test_split

target = heart_clean["malade"]
data = heart_clean.drop("malade", axis=1)

On effectue notre phase d'optimisation avec différentes configuration possible (540)

In [33]:
from sklearn.model_selection import GridSearchCV

parameters = {'solver': ['adam', 'sgd', 'lbfgs'],
              'activation': ['relu', 'logistic'],
              'max_iter': [100, 300, 600, 1000],
              'hidden_layer_sizes':np.arange(5, 15),
              'alpha': [0.0001, 0.001, 0.01]}

clf = GridSearchCV(MLPClassifier(), parameters, n_jobs=-1, verbose=True)

clf.fit(X_train, y_train)

Fitting 5 folds for each of 30 candidates, totalling 150 fits


GridSearchCV(estimator=MLPClassifier(), n_jobs=-1,
             param_grid={'activation': ['relu'], 'alpha': [0.0001],
                         'hidden_layer_sizes': array([ 5,  6,  7,  8,  9, 10, 11, 12, 13, 14]),
                         'max_iter': [100, 300, 600], 'solver': ['adam']},
             verbose=True)

Premier résultats obtenus avec le meilleur estimateurs

In [ ]:
print("Best estimator : {}".format(clf.best_estimator_))
print("Score train : {}".format(clf.score(X_train, y_train)))
print("Score test : {}".format(clf.score(X_test, y_test)))
print("Matrice de confusion :\n  {}".format(classification_report(y_test, clf.predict(X_test))))


In [36]:
clf.best_params_

MLPClassifier(hidden_layer_sizes=6, max_iter=600)


{'activation': 'relu',
 'alpha': 0.0001,
 'hidden_layer_sizes': 6,
 'max_iter': 600,
 'solver': 'adam'}

In [45]:
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2) 

mlp_model = MLPClassifier(random_state=1, max_iter=600, activation="relu", solver="adam", hidden_layer_sizes=6)

mlp_model.fit(X_train, y_train)

MLPClassifier(hidden_layer_sizes=6, max_iter=600, random_state=1)

In [74]:
from sklearn.metrics import confusion_matrix, classification_report
from IPython.display import display

print("Score train : {}".format(mlp_model.score(X_train, y_train)))
print("Score test : {}".format(mlp_model.score(X_test, y_test)))
print("Matrice de confusion :\n  {}".format(classification_report(y_test, mlp_model.predict(X_test))))

Score train : 0.7211645542769264
Score test : 0.7217096192597307
Matrice de confusion :
                precision    recall  f1-score   support

           0       0.71      0.78      0.74      4817
           1       0.74      0.66      0.70      4612

    accuracy                           0.72      9429
   macro avg       0.72      0.72      0.72      9429
weighted avg       0.72      0.72      0.72      9429

